## 一、数据预处理

In [0]:
# 导入依赖包
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [2]:
# 数据处理
tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

batchsize = 512

def preprocess(x, y):  #数据预处理
    x = tf.cast(x, dtype=tf.float32)/ 255. - 0.5
    y = tf.cast(y, dtype=tf.int32)
    return x,y

(x_train, y_train),(x_test, y_test) = keras.datasets.fashion_mnist.load_data()
print(x_train.shape, y_train.shape)

# [b, 28, 28] => [b, 28, 28, 1]
x_train, x_test = np.expand_dims(x_train, axis=3), np.expand_dims(x_test, axis=3)

#训练集预处理
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)) # 构造数据集,这里可以自动的转换为tensor类型了
db_train = db_train.map(preprocess).shuffle(10000).batch(batchsize)

#测试集预处理
db_test = tf.data.Dataset.from_tensor_slices((x_test,y_test)) # 构造数据集
db_test = db_test.map(preprocess).shuffle(10000).batch(batchsize)

db_iter = iter(db_train)
sample = next(db_iter)
print("batch: ", sample[0].shape, sample[1].shape)

(60000, 28, 28) (60000,)
batch:  (512, 28, 28, 1) (512,)


## 二、构建CNN基本单元Conv+BN+Relu类模块
基本的卷积神经网络单元由：卷积层+批量归一化+激活层！


In [0]:
class ConvBNRelu(keras.Model):
  def __init__(self, ch, kernelsz=3, strides=1, padding='same'):
    super(ConvBNRelu, self).__init__()

    self.model = keras.models.Sequential([
        keras.layers.Conv2D(ch, kernelsz, strides=strides, padding=padding), # 卷积
        keras.layers.BatchNormalization(), # 批量归一化
        keras.layers.ReLU() # 激活函数
    ])
      
  def call(self, x, training=None):
    x = self.model(x, training=training)
    return x

三、构建Inception Block模块
![](https://freeshow.oss-cn-beijing.aliyuncs.com/blog/20200421105955.png)

In [0]:
class InceptionBlk(keras.Model):

  def __init__(self, ch, strides=1):
    super(InceptionBlk, self).__init__()
    # channel
    self.ch = ch                                        
    self.strides = strides 								

    self.conv1 = ConvBNRelu(ch, strides=strides) # 构造第1个CNN基本单元
    self.conv2 = ConvBNRelu(ch, kernelsz=3, strides=strides) # 构造第2个CNN基本单元，卷积核大小初始为3*3
    self.conv3_1 = ConvBNRelu(ch, kernelsz=3, strides=strides) # 构造第3_1个CNN基本单元，卷积核大小初始为3*3
    self.conv3_2 = ConvBNRelu(ch, kernelsz=3, strides=1) # 构造第3_2个CNN基本单元，卷积核大小初始为3*3

    self.pool = keras.layers.MaxPooling2D(3, strides=1, padding='same') # 最大池化层，same
    self.pool_conv = ConvBNRelu(ch, strides=strides) # 构造CNN基本单元，卷积核大小初始为3*3

  def call(self, x, training=None):
    x1 = self.conv1(x, training=training)
    x2 = self.conv2(x, training=training)

    x3_1 = self.conv3_1(x, training=training)
    x3_2 = self.conv3_2(x3_1, training=training)

    x4 = self.pool(x)
    x4 = self.pool_conv(x4, training=training)
    # concat along axis=channel
    x = tf.concat([x1, x2, x3_2, x4], axis=3) # 通道数扩充，上图理解。
    return x

## 三、构建Res Block 模块
Res Block 模块继承keras.Model或者keras.Layer都可以

![](https://freeshow.oss-cn-beijing.aliyuncs.com/blog/20200421110851.png)

一个 Res Block包含多个Inception Block,且其中的每个Inception Block维度相同，第一个Inception Block对上一个Res Block进行降维。

下面代码中假设每个Res Block中包含两个Inception Block，第一个Inception Block的strides=2,用于降维，第二个Inception Block的strides=1,保持与第一个Inception Block的维度相同。

每个经过一个Res Block后，channel通道数x2

In [0]:
# Res Block 模块。继承keras.Model或者keras.Layer都可以
class Inception(keras.Model):

  def __init__(self, num_layers, num_classes, init_ch=16, **kwargs):
    super(Inception, self).__init__(**kwargs)

    self.init_ch = init_ch # 初始通道
    self.out_channels = init_ch # 初始输出通道数
    self.num_layers = num_layers # 初始层数，就是多少个res block

    self.conv1 = ConvBNRelu(self.init_ch) # 构造1个CNN基本单元

    self.blocks = keras.models.Sequential(name='dynamic-blocks') # 创建一个Sequential容器对象

    # 创建num_layers个Res Block
    for block_id in range(num_layers):

      # 每个Res Block中包含2个Inception Block
      for layer_id in range(2):	
        # 如果是第一个Inception Block  
        if layer_id == 0:	
          # 则strides=2,进行数据降维  
          block = InceptionBlk(self.out_channels, strides=2) 

        else:	# 如果是第二个Inception Block,则strides=1,维度保持与第一个Inception Block一样
          block = InceptionBlk(self.out_channels, strides=1)

        self.blocks.add(block) # 把block放进容器对象blocks中

      # 添加完一层Res Block后，将通道数扩大为2倍。
      self.out_channels *= 2

    self.avg_pool = keras.layers.GlobalAveragePooling2D()
    self.fc = keras.layers.Dense(num_classes)

  def call(self, x, training=None):
    out = self.conv1(x, training=training)
    out = self.blocks(out, training=training)
    out = self.avg_pool(out)
    out = self.fc(out)

    return out

## 四、构建网络

In [6]:
# 调用Inception
model = Inception(2, 10) # 第1参数为Res Block的数目，第2个参数为类别数；
# derive input shape for every layers.
model.build(input_shape=(None, 28, 28, 1))
model.summary()

Model: "inception"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_bn_relu (ConvBNRelu)    multiple                  224       
_________________________________________________________________
dynamic-blocks (Sequential)  multiple                  292704    
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  1290      
Total params: 294,218
Trainable params: 293,226
Non-trainable params: 992
_________________________________________________________________


## 五、模型训练

In [0]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [8]:
model.fit(db_train, epochs=10, validation_data=db_test, validation_freq=2)

Epoch 1/10
118/118 [==============================] - 8s 68ms/step - loss: 0.5951 - accuracy: 0.7967
Epoch 2/10
118/118 [==============================] - 9s 77ms/step - loss: 0.3078 - accuracy: 0.8916 - val_loss: 2.6047 - val_accuracy: 0.1435
Epoch 3/10
118/118 [==============================] - 8s 67ms/step - loss: 0.2546 - accuracy: 0.9086
Epoch 4/10
118/118 [==============================] - 9s 74ms/step - loss: 0.2177 - accuracy: 0.9219 - val_loss: 0.8816 - val_accuracy: 0.7115
Epoch 5/10
118/118 [==============================] - 8s 67ms/step - loss: 0.1950 - accuracy: 0.9302
Epoch 6/10
118/118 [==============================] - 9s 76ms/step - loss: 0.1737 - accuracy: 0.9370 - val_loss: 0.4438 - val_accuracy: 0.8553
Epoch 7/10
118/118 [==============================] - 8s 67ms/step - loss: 0.1537 - accuracy: 0.9459
Epoch 8/10
118/118 [==============================] - 9s 74ms/step - loss: 0.1365 - accuracy: 0.9509 - val_loss: 0.3366 - val_accuracy: 0.8890
Epoch 9/10
118/118 [====